In [1]:
import os
os.getcwd()

'/Users/vikaslakka/Desktop/FSDS/DeepLearning/Projects/CNN_projects/Deep_classifier/research'

In [2]:
os.chdir('../')
os.getcwd()

'/Users/vikaslakka/Desktop/FSDS/DeepLearning/Projects/CNN_projects/Deep_classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [4]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH,
                    params_filepath= PARAMS_FILE_PATH
                    ):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        create_directories([self.config.artifacts_root])
    
    ## Data ingestion config
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config= self.config.prepare_base_model
        ##Create root directory
        create_directories([config.root_dir])
        
        prepare_base_model_config= PrepareBaseModelConfig(
                root_dir = Path(config.root_dir),
                base_model_path = Path(config.base_model_path),
                updated_base_model_path = Path(config.updated_base_model_path),
                params_image_size= self.params.IMAGE_SIZE,
                params_learning_rate= self.params.LEARNING_RATE,
                params_include_top = self.params.INCLUDE_TOP,
                params_weights= self.params.WEIGHTS,
                params_classes= self.params.CLASSES,
        )
        return prepare_base_model_config

In [21]:
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self,config: PrepareBaseModelConfig):
        self.config = config


    def get_base_model(self):
        """We will get base model from Tensorflow and using transfer learning
            will add our requirements by editing first and last layer of architecture.
        """
        self.model= tf.keras.applications.vgg16.VGG16(
            input_shape= self.config.params_image_size,
            weights= self.config.params_weights,
            include_top= self.config.params_include_top # include only Cnn layer and exclude Ann(Dense network)

        )

        base_model_path= self.config.base_model_path
        self.save_model(path= base_model_path, model= self.model)

    def _prepare_full_model(self, model, classes, freeze_all, freeze_till, learning_rate):
        """
        Preparing full model along with CNN and ANN from an existing model.
        """
        if freeze_all:
            for layer in model.layers:
                model.trainable= False
        elif (freeze_till is not None) and (freeze_till >0):
            for layer in model.layers[:-freeze_till]:
                model.trainable= False
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
                units= classes,
                activation= "softmax",
                )(flatten_in)

        full_model= tf.keras.Model(
                                    model.input,
                                    prediction
        )
        
        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate= learning_rate),
            loss= tf.keras.losses.CategoricalCrossentropy(),
            metrics= ['accuracy']
        )
        full_model.summary()
        return full_model


    def update_base_model(self):
        """Update downloaded method
        """
        self.full_model = self._prepare_full_model(
        model=self.model,
        classes=self.config.params_classes,
        freeze_all=True,
        freeze_till=None,
        learning_rate=self.config.params_learning_rate
        )
        ## Save model
        self.save_model(path= self.config.updated_base_model_path, model= self.full_model)
    @staticmethod # We will define it as static as there is no use to define as method.
    def save_model(path: Path, model: tf.keras.Model):
        """Save updated model
        """
        model.save(path)

In [22]:
try:
    config= ConfigurationManager()
    prepare_base_model_config= config.get_prepare_base_model_config()
    #print(prepare_base_model_config)
    prepare_base_model = PrepareBaseModel(config= prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
    #Sprepare_base_model.save_model()
except Exception as e:
    raise e

[2022-11-22 23:35:40,335: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-11-22 23:35:40,337: INFO: common]: yaml file: params.yaml loaded successfully
[2022-11-22 23:35:40,338: INFO: common]: created directory at: artifacts
[2022-11-22 23:35:40,339: INFO: common]: created directory at: artifacts/prepare_base_model
[2022-11-22 23:35:40,770: WARNING: saving_utils]: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 6

In [ ]:
os.getcwd()

'/Users/vikaslakka/Desktop/FSDS/DeepLearning/Projects/CNN_projects'